In this notebook we want to make a xgboost classification of out 56000 pull request which 10k are non-merged and 46k are merged. we want to do two stage classification and prediction, first stage is for non merged and merged, and then we want to predict and classifiy each category after that


In [59]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from imblearn.over_sampling import SMOTE
import pandas as pd
pd.set_option('display.max_columns', None)
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder



In [69]:
df = pd.read_csv('/Users/amirrshams/Library/CloudStorage/OneDrive-UniversityofWaterloo/Thesis/Dataset/Dataset/sampled_dataset.csv')
df_nonmerged = pd.read_csv('/Users/amirrshams/Library/CloudStorage/OneDrive-UniversityofWaterloo/Thesis/Dataset/Dataset/Non_Merged/Sample/Sample_10000_new_manual.csv')

In [70]:
df.head()

,Unnamed: 0.1,Unnamed: 0,repo_id,status,pr_id,pullreq_id,api_url,url,pr_url,pr_api_url,author_id,author_desc_body,closer_id,comments_counts,comments,commit_counts,code_changes_counts,created_at,closed_at,author_country,closer_country,author_continent,same_country,author_eth,closer_eth,same_eth,prs_white,prs_api,prs_black,prs_hispanic,pri_white,pri_black,pri_api,pri_hispanic,prs_eth_8,prs_eth_7,prs_eth_9,prs_eth_diff,prs_eth_diff_2,prs_pri_same_nationality,prs_experience,prs_succ_rate,prs_popularity,prs_watched_repo,prs_followed_pri,prs_tenure_mnth,prs_main_team_member,repo_pr_tenure_mnth,repo_pr_popularity,repo_pr_team_size,perc_external_contribs,pr_opened_at,pr_files_changed,pr_lines_changed,intra_branch
0,1,2250,6,not-merged,2697422,4651,https://api.github.com/repos/cocos2d/cocos2d-x,https://github.com/cocos2d/cocos2d-x,https://github.com/cocos2d/cocos2d-x/pull/4651,https://api.github.com/repos/cocos2d/cocos2d-x...,CocosRobot,\N,NaN,-1.623582,[],-0.500975,1,2013-12-25 04:04:41,2013-12-25 05:41:54,\N,\N,\N,0,Unknown,API,0,0.889200,0.008001,0.088110,0.014689,0.059421,0.003402,0.933594,0.001673,Unknown,White,Unknown,White,White,0,1.915057,1.150350,-0.444593,0,0,0.077583,0,0.224940,1.028133,0.992756,-0.699726,1387940681,-1.047192,-1.251865,0
1,2,5916,6,not-merged,7456135,11761,https://api.github.com/repos/cocos2d/cocos2d-x,https://github.com/cocos2d/cocos2d-x,https://github.com/cocos2d/cocos2d-x/pull/11761,https://api.github.com/repos/cocos2d/cocos2d-x...,CocosRobot,\N,NaN,-1.623582,[],-0.500975,9,2015-05-08 09:07:16,2015-05-11 01:53:15,\N,china,\N,0,Unknown,Unknown,0,0.889200,0.008001,0.088110,0.014689,0.569895,0.032570,0.367695,0.028239,Unknown,White,Unknown,White,White,0,2.676846,0.764089,-0.161012,0,0,0.488885,0,0.608178,1.292924,1.375811,-0.232106,1431068836,-1.047192,-1.251865,0
2,3,17935,183,not-merged,645270,2808,https://api.github.com/repos/zendframework/zen...,https://github.com/zendframework/zendframework,https://github.com/zendframework/zendframework...,https://api.github.com/repos/zendframework/zen...,blanchonvincent,Usage with static instead of self\n,NaN,-1.623582,[],-0.500975,1,2012-10-19 19:34:23,2012-10-30 19:46:43,france,united states,Europe,0,White,White,1,0.982306,0.003467,0.010741,0.002705,0.930962,0.031869,0.009878,0.025256,White,White,White,White,White,0,0.995788,1.083482,0.968358,1,0,-1.253013,0,-0.069841,1.172173,0.886383,0.179764,1350668063,-1.047192,-1.251865,0
3,4,19620,183,not-merged,2203867,5219,https://api.github.com/repos/zendframework/zen...,https://github.com/zendframework/zendframework,https://github.com/zendframework/zendframework...,https://api.github.com/repos/zendframework/zen...,samsonasik,\N,NaN,0.182242,"['Woah, ancient links spotting :O\n', '@Ocrami...",-0.500975,1,2013-10-03 19:34:04,2013-10-03 19:48:05,indonesia,spain,Asia,0,API,White,0,0.024666,0.975110,0.000109,0.000099,0.996368,0.002142,0.000852,0.000634,API,API,API,API,API,0,1.759672,0.880055,1.906032,1,1,0.279995,0,0.280881,1.227766,1.307105,0.144283,1380821644,-1.047192,-1.251865,0
4,5,27952,340,not-merged,23622497,6647,https://api.github.com/repos/netty/netty,https://github.com/netty/netty,https://github.com/netty/netty/pull/6647,https://api.github.com/repos/netty/netty/pulls...,fenik17,Motivation:\r\n\r\n1. The use of the `Internet...,NaN,-0.720670,['Cherry-picked into 4.1 (970d310ec9ae3d0581ad...,-0.500975,3,2017-04-19 18:56:34,2017-04-20 03:23:50,russia,germany,Asia,0,White,White,1,0.990941,0.000334,0.000025,0.008697,0.992539,0.000581,0.003632,0.002701,White,White,White,White,White,0,0.105514,-1.103914,-0.997446,1,1,-1.017751,0,1.009221,1.516362,1.220494,-0.886764,1492628194,-1.047192,-1.251865,0


# preprocessing
only keep the values that are important to us and are used in the model

In [71]:
#merge the two datasets and only add the manual analysis column
df = pd.concat([df, df_nonmerged['manual_analysis']], axis=1)

In [72]:
#drop all columns except for commit_counts + prs_experience + prs_succ_rate + prs_popularity + prs_followed_pri + prs_watched_repo + prs_tenure_mnth + prs_main_team_member + repo_pr_tenure_mnth + repo_pr_popularity + repo_pr_team_size + perc_external_contribs +intra_branch + pr_files_changed + pr_lines_changed + (1 | repo_id) + (1 | author_id)
df = df[['pr_id', 'commit_counts', 'prs_experience', 'prs_succ_rate', 'prs_popularity', 'prs_followed_pri', 'prs_watched_repo', 'prs_tenure_mnth', 'prs_main_team_member', 'repo_pr_tenure_mnth', 'repo_pr_popularity', 'repo_pr_team_size', 'perc_external_contribs', 'intra_branch', 'pr_files_changed', 'pr_lines_changed', 'repo_id','status', 'manual_analysis']]

In [73]:
#show rows with NaN values
df[df.isna().any(axis=1)]
#how many of the rows in perc_external_contrib are NaN 
df['perc_external_contribs'].isna().sum()
#show where perc_external_contribs and manual_analysis are NaN
df[df['perc_external_contribs'].isna() & df['manual_analysis'].isna()]

,pr_id,commit_counts,prs_experience,prs_succ_rate,prs_popularity,prs_followed_pri,prs_watched_repo,prs_tenure_mnth,prs_main_team_member,repo_pr_tenure_mnth,repo_pr_popularity,repo_pr_team_size,perc_external_contribs,intra_branch,pr_files_changed,pr_lines_changed,repo_id,status,manual_analysis
11811,83434,-0.540434,-1.532904,-1.714328,-1.135762,1,1,0.103728,0,-3.734906,-0.827321,-2.470590,NaN,0,-0.488071,-0.857099,4416,merged,NaN
13035,112871,-0.540434,-1.532904,-1.714328,-1.555667,0,1,-3.555457,0,-3.734906,-1.938174,-2.470590,NaN,0,-1.195553,-1.357043,6286,merged,NaN
14055,550093,-0.540434,-0.724022,0.790595,0.871954,0,0,-0.639497,0,-3.734906,-0.712193,-0.572532,NaN,1,-1.195553,-1.357043,8623,merged,NaN
14486,711063,1.554526,-1.532904,-1.714328,0.631808,1,0,0.043169,0,-2.543178,0.317436,-0.310316,NaN,0,0.219410,0.752076,9703,merged,NaN
14657,279617,-0.540434,-0.375655,0.726262,0.543858,0,1,-0.089816,0,-3.734906,-1.027982,-1.323155,NaN,0,-0.488071,-0.226240,10102,merged,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52719,150397,-0.540434,0.380274,0.775170,0.043060,0,1,-0.701892,0,-2.983010,-1.465002,-2.470590,NaN,0,-0.488071,-0.624638,287583,merged,NaN
52763,196520,-0.540434,-1.532904,-1.714328,-1.135762,0,1,-3.555457,0,0.121783,-1.234648,-2.470590,NaN,0,-0.488071,-0.357155,44939,merged,NaN
52764,1148556,-0.540434,-0.108968,0.881928,0.647964,0,0,0.132722,0,-1.791282,-0.414212,-2.470590,NaN,0,-0.488071,-0.067737,404631,merged,NaN
52814,1762565,-0.540434,-0.980756,-1.714328,-0.580678,0,0,0.160921,0,-1.624066,-1.676416,-2.470590,NaN,0,0.926891,-0.124695,2245795,merged,NaN


In [74]:
#make status column binary, non-merged = 1 and merged = 0
df['status'] = df['status'].replace('merged', 0)
df['status'] = df['status'].replace('not-merged', 1)

In [75]:
#encode the manual_analysis column using one-hot encoding
label_encoder = LabelEncoder()
df['manual_analysis'] = label_encoder.fit_transform(df['manual_analysis'])
#show what is encoded to what

encoded_classes = label_encoder.classes_
encoded_values = label_encoder.transform(label_encoder.classes_)
class_to_number_mapping = dict(zip(encoded_classes, encoded_values))

# Displaying the mapping
print("Category to encoded number mapping:")
for category, num in class_to_number_mapping.items():
    print(f"{category}: {num}")



Category to encoded number mapping:
Chaotic: 0
Duplicate: 1
Merge Conflict: 2
No Comment: 3
No Reason: 4
Not PR: 5
Quality: 6
Replaced: 7
Resolved: 8
Stale: 9
Successful: 10
Unnecessary: 11
nan: 12


In [76]:
df.head(10003)

,pr_id,commit_counts,prs_experience,prs_succ_rate,prs_popularity,prs_followed_pri,prs_watched_repo,prs_tenure_mnth,prs_main_team_member,repo_pr_tenure_mnth,repo_pr_popularity,repo_pr_team_size,perc_external_contribs,intra_branch,pr_files_changed,pr_lines_changed,repo_id,status,manual_analysis
0,2697422,-0.500975,1.915057,1.150350,-0.444593,0,0,0.077583,0,0.224940,1.028133,0.992756,-0.699726,0,-1.047192,-1.251865,6,1,3
1,7456135,-0.500975,2.676846,0.764089,-0.161012,0,0,0.488885,0,0.608178,1.292924,1.375811,-0.232106,0,-1.047192,-1.251865,6,1,3
2,645270,-0.500975,0.995788,1.083482,0.968358,0,1,-1.253013,0,-0.069841,1.172173,0.886383,0.179764,0,-1.047192,-1.251865,183,1,3
3,2203867,-0.500975,1.759672,0.880055,1.906032,1,1,0.279995,0,0.280881,1.227766,1.307105,0.144283,0,-1.047192,-1.251865,183,1,4
4,23622497,-0.500975,0.105514,-1.103914,-0.997446,1,1,-1.017751,0,1.009221,1.516362,1.220494,-0.886764,0,-1.047192,-1.251865,340,1,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9998,1421210,-0.500975,0.009439,0.780038,-0.241580,0,0,0.077583,0,-1.620015,-1.486026,-1.579530,0.284074,0,-0.438619,-0.793484,1380818,1,10
9999,16255814,-0.041089,0.778058,0.342857,1.054699,0,1,0.726456,0,0.628189,1.280257,0.454875,0.214885,0,1.583017,0.973142,35846,1,8
10000,5770123,-0.540434,2.048573,0.404699,-0.470228,0,0,0.266574,0,0.508688,1.392386,1.450083,-0.209722,0,-1.195553,-1.357043,6,0,12
10001,8959829,-0.540434,0.669444,0.358402,1.975478,1,1,-0.242571,0,0.650822,1.525006,1.566186,-0.060082,0,-1.195553,-1.357043,6,0,12


In [ ]:
#in case of using smote
# imputer = SimpleImputer(strategy='mean')  # or 'median', 'most_frequent', etc.
# X_imputed = imputer.fit_transform(X)

# # Apply SMOTE
# smote = SMOTE()
# X_smote, y_smote = smote.fit_resample(X_imputed, y)


## Stage 1

In [77]:
X = df.drop(['status', 'manual_analysis','pr_id','repo_id'], axis=1)
Y = df['status']
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [78]:
model_stage1 = xgb.XGBClassifier()
model_stage1.fit(x_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

In [79]:
predictions_stage1 = model_stage1.predict(x_test)
print("Stage 1 - Classification Report:\n", classification_report(y_test, predictions_stage1))


Stage 1 - Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00      8524
           1       1.00      0.99      1.00      2042

    accuracy                           1.00     10566
   macro avg       1.00      1.00      1.00     10566
weighted avg       1.00      1.00      1.00     10566



In [80]:
full_predictions_stage1 = model_stage1.predict(df.drop(['status', 'manual_analysis', 'pr_id','repo_id'], axis=1))
# Adding predictions back to the dataset
df['predicted_status'] = full_predictions_stage1


In [81]:
df.head(10003)
#show how many instances are predicted_status and status are different
df[df['predicted_status'] != df['status']]

,pr_id,commit_counts,prs_experience,prs_succ_rate,prs_popularity,prs_followed_pri,prs_watched_repo,prs_tenure_mnth,prs_main_team_member,repo_pr_tenure_mnth,repo_pr_popularity,repo_pr_team_size,perc_external_contribs,intra_branch,pr_files_changed,pr_lines_changed,repo_id,status,manual_analysis,predicted_status
385,10710116,2.052480,0.551998,0.377336,-1.415663,0,0,-1.253013,0,0.545840,0.136861,-0.148184,0.035603,1,1.705726,2.109971,9204,1,3,0
902,4109359,1.926326,0.551998,0.730687,0.449347,0,1,0.077583,1,-2.544086,-1.556927,-1.032777,-0.935148,0,3.212820,2.061389,9281863,1,11,0
1034,41500541,2.363751,-0.473401,0.780038,0.968358,0,0,0.587516,0,0.647841,0.815280,0.585820,0.214885,0,6.285401,2.338621,6401040,1,0,0
1100,6390465,2.317450,0.257108,0.305998,1.228275,0,1,0.354925,0,-0.864063,-1.345569,-0.758361,0.179764,0,0.881939,1.108024,7176443,1,10,0
1490,9041941,2.052480,0.400404,0.522617,-0.089947,0,0,0.742459,0,-0.589978,-1.596947,-0.990372,-0.522283,0,2.438667,1.808288,15285703,1,7,0
3265,3817724,2.269178,-0.108146,0.917782,-0.444593,0,0,0.305644,0,-1.127077,-1.151079,-1.243193,-0.192570,0,2.576372,1.891247,8660746,1,8,0
3850,14382193,1.706111,1.118215,0.126238,-0.026376,1,1,0.354925,0,0.686111,-0.433588,-0.499731,-0.076603,0,2.048901,1.119534,62688,1,7,0
4252,19578733,1.531270,1.137150,0.905500,0.083629,0,0,0.967082,0,0.951808,-0.260626,-0.217931,-1.460301,0,1.537983,1.113819,170988,1,10,0
5794,6474200,1.432580,-0.046081,0.636718,0.476167,1,0,-0.181025,0,-0.788819,-1.486026,-1.871830,-4.687270,1,1.537983,1.201711,10982460,1,11,0
6576,16663966,2.408236,1.888922,0.853337,1.220685,1,1,0.928868,0,0.334092,-0.190708,-0.517338,-3.382070,0,2.123141,2.102638,3218707,1,8,0


## Stage 2 with smote

In [89]:
non_merged_data = df[df['predicted_status'] == 1]
X_non_merged = non_merged_data.drop(['status', 'manual_analysis', 'predicted_status', 'pr_id','repo_id'], axis=1)
y_non_merged = non_merged_data['manual_analysis']


In [83]:
#imputing for smote, the nan values are only in perc_external_contribs
imputer = SimpleImputer(strategy='mean')  
X_non_merged_imputed = imputer.fit_transform(X_non_merged)

# Apply SMOTE
smote = SMOTE()
X_train_non_merged_smote, y_train_non_merged_smote = smote.fit_resample(X_non_merged_imputed, y_non_merged)

X_train_non_merged, X_test_non_merged, y_train_non_merged, y_test_non_merged = train_test_split(X_train_non_merged_smote, y_train_non_merged_smote, test_size=0.2, random_state=42)


In [84]:
model_non_merged = xgb.XGBClassifier()
model_non_merged.fit(X_train_non_merged_smote, y_train_non_merged_smote)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              objective='multi:softprob', predictor=None, ...)

In [85]:
predictions_non_merged = model_non_merged.predict(X_test_non_merged)
print("Stage 2 - Classification Report for Non-merged Data:\n", classification_report(y_test_non_merged, predictions_non_merged))


Stage 2 - Classification Report for Non-merged Data:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00       561
           1       0.92      0.93      0.92       518
           2       0.96      0.99      0.97       557
           3       0.94      0.87      0.91       580
           4       0.92      0.93      0.92       547
           5       1.00      1.00      1.00       501
           6       0.93      0.92      0.92       548
           7       0.91      0.87      0.89       535
           8       0.93      0.84      0.88       561
           9       0.94      0.97      0.96       562
          10       0.78      0.91      0.84       565
          11       0.86      0.82      0.84       525

    accuracy                           0.92      6560
   macro avg       0.92      0.92      0.92      6560
weighted avg       0.92      0.92      0.92      6560



## Stage 2 without Smote

In [90]:
X_train_non_merged, X_test_non_merged, y_train_non_merged, y_test_non_merged = train_test_split(X_non_merged, y_non_merged, test_size=0.2, random_state=42)



In [91]:
model_non_merged = xgb.XGBClassifier()
model_non_merged.fit(X_train_non_merged, y_train_non_merged)

# Predictions and evaluation
predictions_non_merged = model_non_merged.predict(X_test_non_merged)
print("Classification Report for Stage 2 - Non-merged Data:\n", classification_report(y_test_non_merged, predictions_non_merged))


Classification Report for Stage 2 - Non-merged Data:
               precision    recall  f1-score   support

           0       0.00      0.00      0.00         5
           1       0.00      0.00      0.00        87
           2       0.00      0.00      0.00        21
           3       0.33      0.18      0.23       181
           4       0.16      0.03      0.05       101
           5       0.00      0.00      0.00         2
           6       0.08      0.02      0.03       146
           7       0.19      0.12      0.15       194
           8       0.19      0.12      0.15       247
           9       0.19      0.05      0.07        65
          10       0.37      0.62      0.46       539
          11       0.25      0.37      0.30       409

    accuracy                           0.29      1997
   macro avg       0.15      0.13      0.12      1997
weighted avg       0.24      0.29      0.25      1997



/Users/amirrshams/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/amirrshams/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/amirrshams/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
